In [ ]:
import cv2 as cv
from cv2 import VideoCapture
import numpy as np

In [ ]:
cv.destroyAllWindows()

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes

In [ ]:
from view_controller import VideoReader
from box_extractor import BoxExtractor

cap = VideoCapture("worms.avi")

video = VideoReader(cap, padding_size=(251, 251))
extractor = BoxExtractor(find_boxes, slice_size=251, object_size=150)

i = 0
while video.next_frame():
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    image = video.get_frame()
    head_coords, rois = extractor.run(image)

    # Draw bboxes
    for box in head_coords:
        x, y, w, h = box
        cv.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 3)

    cv.imshow("BBOXES", image)
    
    print(i)
    i += 1

cap.release()
cv.destroyAllWindows()

In [ ]:
from view_controller import ViewController
from timing_context import TimingContext

cap = VideoCapture("worms.avi")
sim = ViewController(cap, color_conversion=cv.COLOR_BGR2GRAY)

while True:
    with TimingContext():
        sim.next_frame()

    sim.visualize_world()
    sim.move_position(5, 5)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cv.destroyAllWindows()

In [ ]:
import math

fps = 40
micro_view_time = 200
tracking_time = 50
moving_time = 25

frame_time = 1 / fps
micro_view_frames = math.ceil(micro_view_time / frame_time)
tracking_frames = math.ceil(tracking_time / frame_time)
moving_frames = math.ceil(moving_time / frame_time)

while True:
    # Here we assume that the worm is kinda centered on the screen
    sim.micro_view()  # Takes 'micro_frames' frames, can do other stuff meanwhile
    sim.next_frame(micro_view_frames - tracking_frames)

    cam_view = sim.camera_view()
    dx, dy = real_time_tracking(cam_view)  # Takes 'track_frames' frames, can't do anything meanwhile
    sim.next_frame(tracking_frames)  # micro and tracking finished

    sim.next_frame(moving_frames)  # moving takes 'move_frames' frames, can do other stuff meanwhile
    sim.move_position(dx, dy)

In [ ]:
import numpy as np

def find_good_times(head_coords: np.ndarray, thresh: int):
    """
    For each track and for each frame calculates when it's distant enough from all other worms.
    returns good_times, where good_times[i, j] is true if track i on time j is distant enough.
    """
    # head_coords: [num_tracks, time, 2] where 2 represent head coord in (x,y) format
    
    assert head_coords.ndim == 3
    assert head_coords.shape[-1] == 2

    N, T, D = head_coords.shape

    head_coords = head_coords.astype(dtype=np.float32)
    head_coords = np.swapaxes(head_coords, 0, 1)

    coords1 = np.broadcast_to(head_coords.reshape(T, N, 1, D), (T, N, N, D))
    coords2 = np.broadcast_to(head_coords.reshape(T, 1, N, D), (T, N, N, D))

    # diffs[t, a, b] = at time t, worm a coords - worm b coords
    diffs = coords1 - coords2

    # dotprod along last dimension
    dists = np.sum(diffs ** 2, axis=-1, dtype=np.float32)

    # create diagonal nan array
    nans = np.zeros(shape=(N, N), dtype=np.float32)
    np.fill_diagonal(nans, val=np.nan)
    nans = np.broadcast_to(nans.reshape(1, N, N), (T, N, N))

    # set to nan dist between a coord and itself
    dists = dists + nans

    # disregard nans when calculating min
    min_dists = np.nanmin(dists, axis=-1)

    # check where the min_dists is within threshold 
    good_times = np.swapaxes(min_dists, 0, 1) > thresh ** 2 # Times when closest neighbor is larger than threshold

    # returns shape [num_tracks, time]
    return good_times

In [ ]:
arr = np.random.uniform(high=5000, size=(30, 3600 * 40, 2))
N, T, D = arr.shape

res = find_good_times(arr, 200)

frame_nums = np.expand_dims(np.arange(T), 0)
frame_nums = frame_nums.repeat(N, axis=0)
frame_nums[res == False] = -1

print(frame_nums)